# MODEL DEFINITION

In [2]:
import numpy as np
import torch
import torch.nn as nn

In [3]:
class SinusoidalActivation(nn.Module):
  def forward(self,x):
    return torch.sin(2 * np.pi * x)

$$
\begin{cases}
u_t + u  u_x - \left(\frac{0.01}{\pi}\right) u_{xx} = 0, & x \in [-1,1], \, t \in [0,1], \\
u(x, 0) = -\sin(\pi x),  \\
u(-1, t) = u(1, t) = 0,
\end{cases}
$$

In [4]:
class Burgers1D(nn.Module):

############################### NET ARCHITECTURE ###############################

  def __init__(self, num_neurons, num_layers, nu):
    self.nu = nu

    super(Burgers1D, self).__init__()
    self.num_neurons = num_neurons
    self.num_layers = num_layers

### Imput layer, fully connected to the first hidden layer, sinusoidal activation
### function

    layer_list = [nn.Linear(2, self.num_neurons)]
    layer_list.append(SinusoidalActivation())

### Loop for the description of the hidden layers: fully connected layers,
### hiperbolic tangent activation function

    for _ in range (self.num_layers - 2):
      layer_list.append(nn.Linear(self.num_neurons, self.num_neurons))
      layer_list.append(nn.Tanh())

### Output layer, 1 output

    layer_list.append(nn.Linear(self.num_neurons, 1))
    self.layers = nn.ModuleList(layer_list)


################## FEED-FORWARD PROPAGATION ###########################

  def forward(self, x, t):
     """
        Params:
        x - array of shape (N, 1), input x coordinates
        t - array of shape (N, 1), input time coordinate
        Returns:
        u - tensor of shape (N, 1), output x-velocity
        f - x-momentum PDE evaluation of shape (N, 1)

     """
     x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
     t = torch.tensor(t, dtype=torch.float32, requires_grad=True)

     input_data = torch.hstack([x, t])

     out = input_data

     for layer in self.layers:
      out = layer(out)

     u = out[:] # (N, 1) each

     u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
     u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
     u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]

        # Evaluate momentum PDE's

     f = u_t + u * u_x - self.nu * u_xx # (N, 1)


     return u, f

#LOSS FUNCTION

In [5]:
##################### LOSS FUNCTION DESIGN #####################################

class Burgers1DLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.mse = nn.MSELoss()

  def forward(self, u_init, u_net_init, u_net_BC, f):

     L_init = self.mse(u_init, u_net_init)
     L_boundary = self.mse(u_net_BC, torch.zeros_like(u_net_BC))
     L_pde  = self.mse(f, torch.zeros_like(f))

     return L_init + L_boundary + L_pde

#DATA LOADING

In [1]:
import scipy

In [6]:
def boundary_indices(T, N):
    """
    Returns a boolean mask marking the boundary spatial points for all timesteps
    Params:
    N - # of data samples (space locations) in problem
    T - # of timesteps
    Return:
    nd-array of shape (N*T, )
    """
    # Create grid for one timestep
    # Data is a 100 x 256 matrix
    # 296 boundary points out of the 5000 total, this function creates a boolean mask
    # to identify them


    grid = np.zeros((100, 256))

    # Set boundary to 1
    grid[:, 0] = 1
    grid[:, -1] = 1

    grid_in = grid.flatten().reshape(-1,1)

    # Flatten final grid into column vector to be used in training
    boundary_positions = grid_in.astype(bool)

    return boundary_positions


In [7]:
def init_indices(T, N):
    """
    Returns a boolean mask marking the initial condition points
    Params:
    N - # of data samples (space locations) in problem
    T - # of timesteps
    Return:
    nd-array of shape (N*T, )
    """
    # Create grid for one timestep
    # Data consists of a 100 x 256 matrix
    # 256 initial points


    grid = np.zeros((100, 256))

    # Set boundary to 1

    grid[0, :] = 1

    grid_init_in = grid.flatten().reshape(-1,1)

    # Flatten final grid into column vector to be used in training
    init_positions = grid_init_in.astype(bool)

    return init_positions


In [14]:
import csv

In [8]:
def data_loading () :
  x=torch.linspace(-1,1,256)
  t=torch.linspace(0,1,100)
  N, T = x.shape[0], t.shape[0]
  XX,TT = torch.meshgrid(x,t)
  XX = XX.transpose(1,0)
  TT = TT.transpose(1,0)

  x_in = XX.flatten().reshape(-1, 1)  # NT x 1
  t_in = TT.flatten().reshape(-1, 1)  # NT x 1

  return x, t, XX, TT, x_in, t_in, (T, N)

In [15]:
x, t, XX, TT, x_in, t_in, (T, N) = data_loading () # (NT, 1)

# Boundary training points
idx_b = boundary_indices(T, N)
x_boundary = x_in[idx_b, :]
t_boundary = t_in[idx_b, :]
idx_b_train = np.random.choice(x_boundary.shape[0], 30, replace=False)
x_b_train = x_boundary[idx_b_train].reshape(-1,1)
t_b_train = t_boundary[idx_b_train].reshape(-1,1)


# Initial training points
idx_i = init_indices(T ,N)
x_init = x_in[idx_i, :]
idx_i_train = np.random.choice(x_init.shape[0], 30, replace=False)
x_i_train = x_init[idx_i_train].reshape(-1,1)
t_i_train = torch.zeros_like(x_i_train).reshape(-1,1)

u_init = -torch.sin(torch.pi*x_i_train)
u_init = torch.tensor(u_init, dtype = torch.float32)
u_init = u_init.reshape(-1,1)

# Collocation points
samples = int(round(N * T * 0.05))
idx = np.random.choice(x_in.shape[0], samples, replace=False)
x_train = x_in[idx, :]
t_train = t_in[idx, :]

<ipython-input-15-d80e383bbbd0>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u_init = torch.tensor(u_init, dtype = torch.float32)


#Training algorithm

In [13]:
from tqdm import tqdm

In [ ]:
def main(num_neurons, num_layers, epochs):
    """
    Params:
    num_neurons - int, # of hidden units for each neural network layer
    num_layers - int, # of neural network layers
    epochs - int, # of training epochs
    train_selection - float (0,1), fraction of all data (N*T) to select for training OR
                      'BC', select the boundary conditions for training (all timesteps)
    """
    nu = 0.01/torch.pi


    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    criterion = Burgers1DLoss()

    optimizer = torch.optim.LBFGS(PINN_model.parameters(), line_search_fn='strong_wolfe')

    def closure():
        """Define closure function to use with LBFGS optimizer"""
        optimizer.zero_grad()   # Clear gradients from previous iteration

        u_net, f = PINN_model(x_train, t_train)
        u_net_BC, g = PINN_model(x_b_train, t_b_train)
        u_net_init, h = PINN_model(x_i_train, t_i_train)

        loss = criterion(u_init, u_net_init, u_net_BC, f)

        loss.backward() # Backprogation
        return loss

    def training_loop(epochs):
        """Run training loop"""

        for i in tqdm(range(epochs), desc='Training epochs: '):


          optimizer.step(closure)
          loss = closure().item()

    training_loop(epochs=epochs)

    # Save trained model parameters
    torch.save(PINN_model.state_dict(), 'trained_model_name.pth')
    return

#Model evaluation

In [ ]:
if __name__ == '__main__':
    main(num_neurons=num_neurons, num_layers=num_layers, epochs=epochs)

Training epochs:   0%|          | 0/100 [00:00<?, ?it/s]<ipython-input-3-dc21a98126ce>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
<ipython-input-3-dc21a98126ce>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t, dtype=torch.float32, requires_grad=True)
Training epochs: 100%|██████████| 100/100 [01:08<00:00,  1.45it/s]


In [16]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

# Accuracy dependence on the number of collacation points used in training systematic study

In [ ]:
def load_saved_model_20_5(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/20_5.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

def load_saved_model_20_25(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/20_25.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

def load_saved_model_20_45(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/20_45.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

def load_saved_model_60_5(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/60_5.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

def load_saved_model_60_25(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/60_25.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

def load_saved_model_60_45(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/60_45.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

def load_saved_model_100_5(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/100_5.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

def load_saved_model_100_25(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/100_25.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

def load_saved_model_100_45(num_layers, num_neurons):

    nu = 0.01

    # load saved state_dict
    PINN_model = Burgers1D(num_neurons=num_neurons, num_layers=num_layers, nu=nu)
    PINN_model.load_state_dict(torch.load('/content/100_45.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
# Model loading

#10 initial points // 10 boundary points // 5/25/45 collocation points
model_20_5  = load_saved_model_20_5(num_layers = 9, num_neurons = 20)
model_20_25 = load_saved_model_20_25(num_layers = 9, num_neurons = 20)
model_20_45 = load_saved_model_20_45(num_layers = 9, num_neurons = 20)

#30 initial points // 30 boundary points // 5/25/45 collocation points
model_60_5  = load_saved_model_60_5(num_layers = 9, num_neurons = 20)
model_60_25 = load_saved_model_60_25(num_layers = 9, num_neurons = 20)
model_60_45 = load_saved_model_60_45(num_layers = 9, num_neurons = 20)

#50 initial points // 50 boundary points // 5/25/45 collocation points
model_100_5  = load_saved_model_100_5(num_layers = 9, num_neurons = 20)
model_100_25 = load_saved_model_100_25(num_layers = 9, num_neurons = 20)
model_100_45 = load_saved_model_100_45(num_layers = 9, num_neurons = 20)

In [ ]:
#Evaluation of the different models
#u_n1_n2 is a tensor containing the predicted solution across the spatio-temporal domain
#f_n1_n2 is a tensor containing the evaluation of the Burgers equation with respect to the predicted solution
#UU_n1_n2 is a reshape of the solution for its representation

u_20_5, f_20_5 = model_20_5(x_in, t_in)
UU_20_5 = u_20_5.reshape(T, N).detach().transpose(1,0)

u_20_25, f_20_25 = model_20_25(x_in, t_in)
UU_20_25 = u_20_25.reshape(T, N).detach().transpose(1,0)

u_20_45, f_20_45 = model_20_45(x_in, t_in)
UU_20_45 = u_20_45.reshape(T, N).detach().transpose(1,0)

u_60_5, f_60_5 = model_60_5(x_in, t_in)
UU_60_5 = u_60_5.reshape(T, N).detach().transpose(1,0)

u_60_25, f_60_25 = model_60_25(x_in, t_in)
UU_60_25 = u_60_25.reshape(T, N).detach().transpose(1,0)

u_60_45, f_60_45 = model_60_45(x_in, t_in)
UU_60_45 = u_60_45.reshape(T, N).detach().transpose(1,0)

u_100_5, f_100_5 = model_100_5(x_in, t_in)
UU_100_5 = u_100_5.reshape(T, N).detach().transpose(1,0)

u_100_25, f_100_25 = model_100_25(x_in, t_in)
UU_100_25 = u_100_25.reshape(T, N).detach().transpose(1,0)

u_100_45, f_100_45 = model_100_45(x_in, t_in)
UU_100_45 = u_100_45.reshape(T, N).detach().transpose(1,0)

<ipython-input-3-dc21a98126ce>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
<ipython-input-3-dc21a98126ce>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t, dtype=torch.float32, requires_grad=True)


In [ ]:
XX_p=XX.detach().transpose(1,0)
TT_p=TT.detach().transpose(1,0)

#Exact solution

In [ ]:
def data_loading_2 ():

  data = scipy.io.loadmat('/content/burgers_equation.mat')
  UU_star = torch.tensor(data['usol']) #        256 x 100
  X_star = data['x']                   #        256 x 1
  t_star = data['t']                   #        100  x 1

  N, T= X_star.shape[0], t_star.shape[0]

  # Column vector transformation  25600 x 1
  u_star_in = UU_star.flatten().reshape(-1, 1)

  return UU_star, u_star_in


In [ ]:
UU_star, u_star_in = data_loading_2 ()

#Mean squared error

In [ ]:
def MSE(UU):

    # Initialize array to hold sum of square errors (sse)
    N = UU.shape[0]
    T = UU.shape[1]

    sse = np.zeros(T)

    for i in range(T):
        # Prediction for time i
        sse[i] = ((UU[:,i]-UU_star[:,i])**2).sum()

    # Sum sse over timesteps
    sse = sse.sum()

    # Average sse over all samples and all timesteps to get mean squared error (mse)
    mse = sse / (N * T)

    #R^2 coefficient calculation
    # Find total sum of squares (SSTO)
    ssto = ((u_star_in - u_star_in.mean()) ** 2).sum()
    ssto = ssto.detach()
    R2 = (1 - (sse / ssto))

    return mse, R2.detach()

In [ ]:
# Calculate mse
Nu_list_5 = [20, 60, 100]
U_list_5 = [UU_20_5, UU_60_5, UU_100_5]

rmse_list_5, r2_list_5 = [], []
for i, layer in enumerate(Nu_list_5):
    rmse_i, r2_i = MSE(U_list_5[i]) #each of shape (1, )
    rmse_list_5.append(rmse_i)
    r2_list_5.append(r2_i)

rmse_5, r2_5 = np.vstack(rmse_list_5), np.vstack(r2_list_5) # # epochs tested, u
rmse_df_5, r2_df_5 = pd.DataFrame(rmse_5, columns=['1280 collocation points'], index=Nu_list_5), pd.DataFrame(r2_5, columns=['1280 collocation points'], index=Nu_list_5)

In [ ]:
rmse_df_5

,1280 collocation points
20,0.066125
60,0.020914
100,0.011649


In [ ]:
# Calculate rmse
Nu_list_25 = [20, 60, 100]
U_list_25 = [UU_20_25, UU_60_25, UU_100_25]

rmse_list_25, r2_list_25 = [], []
for i, layer in enumerate(Nu_list_25):
    rmse_i, r2_i = MSE(U_list_25[i]) #each of shape (1, )
    rmse_list_25.append(rmse_i)
    r2_list_25.append(r2_i)

rmse_25, r2_25 = np.vstack(rmse_list_25), np.vstack(r2_list_25) # # epochs tested, u
rmse_df_25, r2_df_25 = pd.DataFrame(rmse_25, columns=['6400 collocation points'], index=Nu_list_25), pd.DataFrame(r2_25, columns=['6400 collocation points'], index=Nu_list_25)

In [ ]:
rmse_df_25

,6400 collocation points
20,0.039617
60,0.003825
100,0.000372


In [ ]:
# Calculate rmse
Nu_list_45 = [20, 60, 100]
U_list_45 = [UU_20_45, UU_60_45, UU_100_45]

rmse_list_45, r2_list_45 = [], []
for i, layer in enumerate(Nu_list_45):
    rmse_i, r2_i = MSE(U_list_45[i]) #each of shape (1, )
    rmse_list_45.append(rmse_i)
    r2_list_45.append(r2_i)

rmse_45, r2_45 = np.vstack(rmse_list_45), np.vstack(r2_list_45) # # epochs tested, u
rmse_df_45, r2_df_45 = pd.DataFrame(rmse_45, columns=['11520 collocation points'], index=Nu_list_45), pd.DataFrame(r2_45, columns=['11520 collocation points'], index=Nu_list_45)

In [ ]:
rmse_df_45

,11520 collocation points
20,0.022891
60,0.000412
100,0.000023


#Results table

In [ ]:
df_rmse = pd.concat([rmse_df_5, rmse_df_25, rmse_df_45], axis=1)
df_rmse

,1280 collocation points,6400 collocation points,11520 collocation points
20,0.066125,0.039617,0.022891
60,0.020914,0.003825,0.000412
100,0.011649,0.000372,0.000023


In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
df_rmse.to_csv('estudiocolocationdef.csv', index=True)
files.download('estudiocolocationdef.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>